In [1]:
clean_up = True
%run StdPackages.ipynb

## Example A:
1. Define global settings that are applied across all modules.
2. Define production structure using nesting trees.
3. Define production module from steps 1-2.
4. Initialize and compile:
    1. Initialize variables (if needed),
    2. Initialize groups definition (add to ```self.s.groups```)
    3. Define argument defining blocks of equations.
    4. Specify settings for different states of the model

*1. Globals:*

Start by defining the global settings: In this section, we draw on the ```SmallOpen``` type. Beyond some global parameters (e.g. interest rate, long run growth rate, etc.), this includes some definitions for time:

In [2]:
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,5)})

*2. Nesting tree:*

We consider a two-sector model $(s1,s2)$ that produces three goods $(a,b,c)$ using inputs $(K,L,M)$. Sector 1 produces  a single using a nested CES, sector 2 produces two outputs using a nested CES/MNL type function.

In [3]:
name = 'A'
data_str = os.path.join(d['data'],'Production_A.xlsx')
read_trees = {'Tree1': {'f':'CES'}, 'Tree2': {'f': 'MNL_out'}} # Keys refer sheet, f refers to type.
Tree = nestingTree.aggTree_from_data(data_str, read_trees = read_trees, name = name)() # apply call function.

*3. Initialize production module from tree*

In [4]:
P = CGE_Production.Production(tree = Tree, glob = glob)

#### 4: Initialize and compile

The method ```self.compile(order=None, initDB=False)``` is defined for all ```GmsPython``` instances, of which ```CGE_Production.Production``` is a specific instance. This method works through two/three steps:
1. ```self.compile_groups:``` Updates the settings ```self.s``` specification of groups from the method ```self.groups()```. A standard method for ```self.groups``` collects ```self.groups()``` for all the modules in ```self.m``` that has such a method defined. The production module has a specific method implemented here. Next, the groups are compiled using ```self.s.Compile.run()```.
2. ```self.compile_states:``` Collects and sorts 'args' for the model stored in ```self.s['args']``` specificied by the method ```self.args()```. A standard method ```self.args``` collects ```self.args()``` for all the modules in ```self.m``` that has such a method defined. The production module has a specific method implemented here.
3. ```self.initDB```: If ```initDB=True``` a method that supplies simple initial values for relevant variables are applied. A standard method is applied that collects ```self.initDB``` methods from the modules in ```self.m```. 

In [5]:
P.compile(initDB=True)

#### 5: Write

In [6]:
P.write();

#### 6: Run
We can set up and run the model in two ways: By specifying a model and parsing it, or asking for it to be initialized and run. We specify which repo to write the data to

In [7]:
# 1: set up model and pass it:
# model = GmsPy.GmsModel(ws=d['work'], **{'cns': 'CONOPT4'})
# P.run(model=model, exportTo = d['work'])
# 2: This performs the same in one:
model = P.run(exportTo = d['work'], ws=d['work'],**{'cns': 'CONOPT4'})